# Disclosure Keywords Related to Pandemic


Reference: https://towardsdatascience.com/keyword-extraction-with-bert-724efca412ea


## Dependicies


In [ ]:
#! pip install transformers
#! pip install torchinfo

! pip install  sentence-transformers

! pip install textacy

! pip install spacy

! python -m spacy download en_core_web_sm

     |████████████████████████████████| 85 kB 3.6 MB/s eta 0:00:011
     |████████████████████████████████| 2.6 MB 6.1 MB/s eta 0:00:01
     |████████████████████████████████| 13.9 MB 17.4 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 42.6 MB/s eta 0:00:01
     |████████████████████████████████| 43 kB 2.8 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 34.0 MB/s eta 0:00:01
  Created wheel for sentence-transformers: filename=sentence_transformers-2.0.0-py3-none-any.whl size=126710 sha256=fe4759a08533bbb719ad86eb23c05c4c5684098e12be0699522d17d8af28b799
  Stored in directory: /Users/zengxu/Library/Caches/pip/wheels/8c/b7/50/451c9a52a337aac5521dbc10544a69e1447d28012feba30742
Successfully built sentence-transformers
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.9.4
    Uninstalling tokenizers-0.9.4:
      Successfully uninstalled tokenizers-0.9.4
  Attempting uninstall: transformers
    Found existing installation: transf

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
import os
import sys
import time
import copy
import glob
import pickle
import random
import datetime
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix, f1_score, accuracy_score, precision_recall_fscore_support
from sklearn.utils import shuffle

from google.colab import drive

import spacy
from textacy.extract import keyterms

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity




ModuleNotFoundError: No module named 'tensorflow'

## Set up

Mount drive & Path:

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# set up data path
import sys, os

cur_path = os.path.join('/content/drive/MyDrive/ColabNotebooks')

sys.path.insert(0, cur_path)

os.chdir(cur_path)
print(os.getcwd())

/content/drive/MyDrive/ColabNotebooks


GPU info

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Thu Aug  5 12:04:29 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch
import torchvision

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
#! pip install torchvision 


In [ ]:
#import shutil

#for f in glob.glob("../FastSR_torch_100_25/*.py"):
#  filename = os.path.basename(f)
#  print(filename)
#  shutil.copyfile(f, os.path.join("../FastSR_CNN_30", filename))

In [ ]:
os.getcwd()

'/content/drive/MyDrive/ColabNotebooks'

# Load Data

In [ ]:
data = pd.read_excel("10K_2020.xlsx",index_col=0).dropna()
data["para_keywords"] = data.para_keywords.apply(lambda x: x.replace("\n",""))
data["para_keywords"] = data.para_keywords.apply(lambda x: x.replace("\xa0",""))
data["para_keywords"] = data.para_keywords.apply(lambda x: x.replace("---------------------------",""))
data.head()

,path,para_keywords
0,D:\10-K\10-K\20\0000001750\0001104659-20-08531...,"Upon entering the fourth quarter in March, we ..."
1,D:\10-K\10-K\20\0000001800\0001104659-20-02390...,"2013 to 2015— President, Early Technologies, C..."
2,D:\10-K\10-K\20\0000001961\0001264931-20-00005...,"After all of our operations were spun off,our ..."
3,D:\10-K\10-K\20\0000002098\0001564590-20-01085...,"If a pandemic, epidemic or outbreak of an infe..."
4,D:\10-K\10-K\20\0000002186\0001654954-20-00224...,"InDecember 2019, a strain of the coronavirus s..."


# Load Model and functions


In [ ]:
model = SentenceTransformer('distilbert-base-nli-mean-tokens')

model = SentenceTransformer('sentence-transformers/paraphrase-xlm-r-multilingual-v1')

NameError: name 'SentenceTransformer' is not defined

In [ ]:
import numpy as np

def mmr(doc_embedding, word_embeddings, words, top_n, diversity):

    # Extract similarity within words, and between words and the document
    word_doc_similarity = cosine_similarity(word_embeddings, doc_embedding)
    word_similarity = cosine_similarity(word_embeddings)

    # Initialize candidates and already choose best keyword/keyphras
    keywords_idx = [np.argmax(word_doc_similarity)]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    for _ in range(top_n - 1):
        # Extract similarities within candidates and
        # between candidates and selected keywords/phrases
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # Calculate MMR
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # Update keywords & candidates
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

In [ ]:
import numpy as np
import itertools

def max_sum_sim(doc_embedding, word_embeddings, words, top_n, nr_candidates):
    # Calculate distances and extract keywords
    distances = cosine_similarity(doc_embedding, word_embeddings)
    distances_candidates = cosine_similarity(word_embeddings, 
                                            word_embeddings)

    # Get top_n words as candidates based on cosine similarity
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # Calculate the combination of words that are the least similar to each other
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

# Seed list


In [ ]:
seed_financial=["raise additional capital","obtain financing","reduce costs","preserve financial flexibility",
                "sharing contribution","credit reduced","reduce operating expenses","pay salaries","largely maintained case",
                "spend reducing compensation","preserve flexibility","sell shares","focus our portfolio"]

In [ ]:
seed_impact=["exacerbate other risks discussed","absences lower productivity","commercial customer decrease","cargo services reduced","operating loss","disruptions supply","temporarily suspended","absence formal restrictions",
             "economic disruption","impact of change","affect demand airline","damage employee relations","extent pandemic prolong","foreign governments deducted","formal restrictions","employees absence","impacts demand services",
             "negatively impacts demand","economic and market uncertainty","flying overall decline","reduced profitability","tax credits","experienced increase sales","affected travel demand","affect business operating","unable quickly reassemble",
             "continued strength","aftermarket growth trends","disruption fiscal",'delay clinical development',"worker absence","affacted travel demand"]

# Sentence matching

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
doc = nlp(data["para_keywords"].iloc[0])
sents = []
for sent in doc.sents:
  sents.append(sent.text)

In [ ]:
candidate_embeddings = model.encode(sents)
doc_embedding = model.encode(["stoppage","employment subsidies","increase important overall","seek work demand","adjusting business plan",
                "deferred arrangements","support services prolonged","sharing contribution","changes certain assumptions",
                "credit reduced","retain sufficient","reduce demand","decrease commercial","predict the impact","commercial operators evaluate",
                "pay salaries","focus our portfolio","portfolio core services","Services segment give","require considerable","temporarily suspend",
                "increase evaluate sensitivity","assessing impact changes","restrictions employees disruptions","increasing discount rates","defense customers believe",
                "quickly reassemble","believe long","maintenance facilities retain","measure maintain","preserve flexibility liquidity","largely maintained cash",
                "freeze new","additional amounts precautionary","evaluate their structure","retain these contract","adopted policies prohibiting","furloughs reduction force",
                "portion furloughed","employees absence formal","worker absence","draw remaining available","mitigate demand variability","segment commercial customers",
                "relief and stimulus","provide relief stimulus","decrease material","capture customer demand","actions reduce","workforce subsequently retain",
                "maintain good relationships","rates adjusting business","decided sell","actions mitigate impact","exit joint venture","exit underperforming","facilities mitigate demand",
                "quickly reassemble skilled","provide unique skill","skilled personnel return","demand airline cargo","lower productivity","preserve flexibility",
                "ability provide aftermarket","decided sell composites","closure airframe maintenance","require considerable management","spend reducing compensation",
                "sets necessary certain","reduction force closure","demand travel behavior","demand variability customers"])
print(candidate_embeddings.shape)
doc_embedding.shape

(83, 768)


(72, 768)

In [ ]:
top_n = 20
distances = cosine_similarity(doc_embedding, candidate_embeddings)

In [ ]:
for index in distances.argsort()[0, ::-1][:top_n]:
  print(distances[0, index], sents[index])

0.3065427 There are no other terms or covenants limiting the availability of this facility.
0.2578585 Government Actions to Mitigate COVID-19’s
0.24818626 Should the demand for skilled personnel return to pre-COVID-19 levels, and if we are unable to quickly reassemble our skilled workforce and subsequently retain a sufficient number of skilled personnel, or we experience a significant or prolonged work stoppage in such an environment, our ability to secure new work and our results of operations and financial condition could be adversely affected.
0.24490565 In the fourth quarter of fiscal 2020, we elected to draw down our Revolving Credit Facility as a precautionary measure in light of economic and market uncertainty presented by COVID-19.
0.24490565 In the fourth quarter of fiscal 2020, we elected to draw down our Revolving Credit Facility as a precautionary measure in light of economic and market uncertainty presented by COVID-19.
0.2437941 In response to the impact from COVID-19, we

# Keywords Matching

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

n_gram_range = (2,5)
stop_words = "english"

# Extract candidate words/phrases
count = CountVectorizer(ngram_range=n_gram_range).fit([data["para_keywords"].iloc[0]])
candidates = count.get_feature_names()

#drop stopwords
#count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([data["para_keywords"].iloc[0]])
#candidates = count.get_feature_names()

In [ ]:
candidates[0:6]

['10 year',
 '10 year senior',
 '10 year senior unsecured',
 '10 year senior unsecured promissory',
 '11 million',
 '11 million in']

In [ ]:
def extract_keyphrase_ngram(top_n):
    seedphrase = seed_financial
             
    mss_ngram = set()
    mmr_ngram = set()
    similarity_ngram = set()
    candidate_embeddings = model.encode(candidates)
    for i in seedphrase:
        doc_embedding = model.encode([i])
        distances = cosine_similarity(doc_embedding, candidate_embeddings)

    #try different parameters
    max_i = max_sum_sim(doc_embedding, candidate_embeddings, candidates, 5, 15)
    #top_n -> try diff parameters
    mmr_i = mmr(doc_embedding, candidate_embeddings, candidates, top_n=10, diversity=0.5)
    mss_ngram = mss_ngram.union(set(max_i))
    mmr_ngram = mmr_ngram.union(set(mmr_i))

    #for index in distances.argsort()[0, ::-1][:top_n]:
    #  similarity_ngram.add(candidates[index])

  mss_mmr_similarity=mss_ngram.union(mmr_ngram)
  
  return mss_mmr_similarity, distances

In [ ]:
mss_mmr_similarity, distances= extract_keyphrase_ngram(20)


NameError: name 'model' is not defined

In [ ]:
mss_mmr_similarity

{'10 year senior unsecured',
 '2020 we decided to',
 '444 million',
 'ability to secure new',
 'account for the concessions as',
 'actions to preserve flexibility in',
 'actions to reduce fixed costs',
 'additional funding',
 'additional funding remaining',
 'affected travel demand',
 'airline cargo services reduced',
 'also evaluate the sensitivity',
 'also retained',
 'and retain',
 'and retain the',
 'are largely',
 'areas the payroll',
 'be direct',
 'be used to pay for',
 'being maintained',
 'business plan assumptions including',
 'by financing',
 'by financing activities continuing operations',
 'capture the customer',
 'cares act',
 'cargo services reduced numbers',
 'cash used',
 'certain payroll',
 'commercial customers decreased',
 'compensation and',
 'compensation and benefits furloughs reduction',
 'continued strength',
 'contributions to',
 'contributions to the',
 'cost of',
 'costs and',
 'costs and overhead which',
 'credit which',
 'credit which reduced',
 'customers

In [ ]:
#save to csv file
df = pd.DataFrame(list(mss_mmr_similarity), columns=['mss_mmr_similarity_ngram'])
df.to_csv("809stopwordsmss_mmr_similarity_ngram(2020).csv")

In [ ]:
len(mss_mmr_similarity)

332

In [ ]:
print(mss_mmr_similarity)

{'demand variability', 'supply chain', 'subsidiary', 'limit', 'source', 'chain and business', 'demand', 'carryback'}


# Keyterm Matching

In [ ]:
doc = nlp(data["para_keywords"].iloc[0])
results = keyterms.yake(doc, ngrams=(1,2,3,4,5), topn=1000, include_pos = None, window_size = 5)
#keyterms.textrank(doc,  topn=20,  include_pos = None, window_size = 3)
candidates, scores = zip(*results)

candidates= list(candidates)

In [ ]:
len(candidates)

1000

In [ ]:
seedlist = ['raise additional capital', 'obtain financing', 'reduce costs', 'preserve financial flexibility', 'obtain clinical supplies', 'reduce operating expenses',
            'obtain regulatory approval','exacerbate other risks discussed','obtain sufficient materials','reduce expenses','require additional resources'
            'obtain components', 'develop new', 'obtain supplies', 'attract new customers','perform critical functions', 'remote working arrangements','reduce infection',
            'avoid large gatherings', 'local governments react', 'provide economic relief', 'pay dividends', 'reduce staffing', 'retain key employees', 'reduce operations'
            , 'impose travel restrictions', 'improve profitability', 'find alternate sources', 'maintain social distancing', 'ensure business continuity'
            , 'sell shares', 'reduce production levels', 'identify trends', 'develop effective treatments', 'extend credit', 'ensure uninterrupted service', 'delay clinical development']

In [ ]:
def extract_keyphrase_keyterm(top_n):
  seedphrase = total_seedphrases

  mss_keyterm = set()
  mmr_keyterm = set()
  similarity_keyterm = set()           
  candidate_embeddings = model.encode(list(candidates))
  for i in seedphrase:
    doc_embedding = model.encode([i])
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    max_i2 = max_sum_sim(doc_embedding, candidate_embeddings, candidates, 20, 20)
    mmr_i2 = mmr(doc_embedding, candidate_embeddings, candidates, top_n=20, diversity=0.5)
    mss_keyterm = mss_keyterm.union(set(max_i2))
    mmr_keyterm = mmr_keyterm.union(set(mmr_i2))

    for index in distances.argsort()[0, ::-1][:top_n]:
      similarity_keyterm.add(candidates[index])
    
  mss_mmr_similarity_keyterm=mss_keyterm.union(mmr_keyterm).union(similarity_keyterm)

    
  return mss_mmr_similarity_keyterm

In [ ]:
mss_mmr_similarity_keyterm = extract_keyphrase_keyterm(20)

In [ ]:
#save to csv file
df1 = pd.DataFrame(list(mss_mmr_similarity_keyterm), columns=['mss_mmr_similarity_keyterm'])
df1.to_csv("2mss_mmr_similarity_keyterm(2020).csv")

### Combine the result of n-gram and keyterm matching(drop duplications)

In [ ]:
mss_mmr_similarity_2020 = mss_mmr_similarity_keyterm.union(mss_mmr_similarity)

In [ ]:
len(mss_mmr_similarity_2020)

2993

In [ ]:
#save to csv file
df1 = pd.DataFrame(list(mss_mmr_similarity_2020), columns=['mss_mmr_similarity_2020'])
df1.to_csv("2mss_mmr_similarity_bothmethod(2020).csv")

mss_mmr_similarity_2020

{'agreement cover',
 'an income approach',
 'other areas the payroll',
 'in demand from commercial airline',
 'uncertainty in certain',
 'actions to preserve',
 'cash and cash equivalent',
 'States',
 'government and defense customers',
 'grant',
 'defense',
 'and/or adjust',
 'follow',
 'business',
 'consolidated balance sheet in fiscal',
 'effective',
 'essential spend',
 'formal',
 'our skilled workforce',
 'to receive 57',
 'assessing the impact of',
 'also exited underperforming contracts',
 'million will be direct grant',
 'as forecasts used',
 'for skilled personnel return',
 'to pay for',
 'maintenance facilities we retain these',
 '10 year senior',
 'of credit which',
 'substantially',
 'evaluate their structure our',
 'have declared states',
 'employees will not damage',
 'reporting units based on',
 'exit our joint venture investment',
 'companies in mitigating the financial',
 'additional funding remain',
 'or covenants limiting',
 'financial impact',
 'the long term we exp